# NN Playground

This notebook will be testing out variations on different neural networks, utilizing various forms of cells/nodes.

Cell types include:
-Backfed input Cell
-Input Cell
-Noisy Input Cell

# Imports

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Base NN Class

Creating a BaseNN class intended to use inheritance in later implementations of different NN's when abstracting the base class to make specialized classes.

In [2]:
# Base class for neural networks
class BaseNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BaseNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

    def forward(self, x):
        raise NotImplementedError("forward method must be implemented in derived classes")

In [3]:
# Instantiate the neural networks
input_size = 2
hidden_size = 3

# Basic Classes

## Perceptron (P)

In [4]:
class Perceptron:
    def __init__(self, input_size):
        # Initialize weights and bias randomly
        self.weights = np.random.rand(input_size)
        self.bias = np.random.rand()

    def activate(self, x):
        # Simple step function as activation
        return 1 if x > 0 else 0

    def forward(self, inputs):
        # Calculate the weighted sum of inputs
        weighted_sum = np.dot(inputs, self.weights) + self.bias

        # Apply the activation function
        output = self.activate(weighted_sum)

        return output

In [5]:
# Example Usage
if __name__ == "__main__":
    # Create a perceptron with 2 input cells
    perceptron = Perceptron(input_size=2)

    # Example input
    input_data = np.array([0.5, 0.8])

    # Get the output from the perceptron
    output = perceptron.forward(input_data)

    print(f"Input: {input_data}")
    print(f"Output: {output}")

Input: [0.5 0.8]
Output: 1


## Feed Forward (FF)

In [6]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(FeedforwardNN, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Linear(hidden_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, 1)  # Single output neuron

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        x = torch.relu(self.hidden_layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Instantiate the neural network
input_size = 2  # Number of input features 
hidden_size = 3  # Number of neurons in the hidden layers
model = FeedforwardNN(input_size, hidden_size)

In [7]:
# Define a sample input
sample_input = torch.tensor([[0.5, 0.3]])  # Example Data

# Forward pass to get the output
output = model(sample_input)

# Print the model architecture and output
print(model)
print("Output:", output.item())

FeedforwardNN(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layer): Linear(in_features=3, out_features=3, bias=True)
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)
Output: 0.4823249578475952


# Radial Basis Network (RBF)

In [8]:
class RadialBasisFunction:
    def __init__(self, input_size, num_centers):
        # Initialize centers and width parameters randomly
        self.centers = np.random.rand(num_centers, input_size)
        self.width = np.random.rand()
        self.weights = np.random.rand(num_centers)
    
    def gaussian(self, x, center, width):
        # Gaussian activation function
        return np.exp(-np.sum((x - center)**2) / (2 * width**2))
    
    def forward(self, inputs):
        # Calculate the activation for each center
        activations = np.array([self.gaussian(inputs, center, self.width) for center in self.centers])
        
        # Calculate the weighted sum of activations
        weighted_sum = np.dot(activations, self.weights)
        
        # Apply a threshold for binary output
        output = 1 if weighted_sum > 0.5 else 0
        
        return output

In [9]:
# Example Usage
if __name__ == "__main__":
    # Create an RBF network with 2 input cells and 3 centers
    rbf_network = RadialBasisFunction(input_size=2, num_centers=3)
    
    # Example input
    input_data = np.array([0.5, 0.8])
    
    # Get the output from the RBF network
    output = rbf_network.forward(input_data)
    
    print(f"Input: {input_data}")
    print(f"Output: {output}")

Input: [0.5 0.8]
Output: 0


# Recurrent Neural Network (RNN)

In [10]:
class SimpleRNN(BaseNN):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__(input_size, hidden_size, output_size=1)
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.recurrent_layer = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, self.output_size)

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        h_t, _ = self.recurrent_layer(x)
        output = torch.sigmoid(self.output_layer(h_t[:, -1, :]))  # Taking the output from the last time step
        return output

## Comparing FF & RNN

In [11]:
# Instantiate the models
feedforward_model = FeedforwardNN(input_size, hidden_size)
simple_rnn_model = SimpleRNN(input_size, hidden_size)

# Forward pass for the feedforward model
sample_input = torch.tensor([[0.5, 0.3]])
output_feedforward = feedforward_model(sample_input)

# Forward pass for the simple RNN model
sample_input_rnn = torch.rand((1, 4, input_size))
output_rnn = simple_rnn_model(sample_input_rnn)

# Print the model architectures and outputs
print("Feedforward Model:")
print(feedforward_model)
print("Output:", output_feedforward.item())

print("\nSimple RNN Model:")
print(simple_rnn_model)
print("Output:", output_rnn.item())

Feedforward Model:
FeedforwardNN(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (hidden_layer): Linear(in_features=3, out_features=3, bias=True)
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)
Output: 0.5554649829864502

Simple RNN Model:
SimpleRNN(
  (input_layer): Linear(in_features=2, out_features=3, bias=True)
  (recurrent_layer): RNN(3, 3, batch_first=True)
  (output_layer): Linear(in_features=3, out_features=1, bias=True)
)
Output: 0.36250656843185425


# Deep Feed Forward (DFF)

In [12]:
# Deep Feed Forward Neural Network
class DeepFeedforwardNN(BaseNN):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeepFeedforwardNN, self).__init__(input_size, hidden_size, output_size)
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_size, hidden_size) for _ in range(2)  # Two hidden layers with 4 nodes each
        ])
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

In [13]:
# Instantiate the deep feedforward neural network
input_size = 3  # Number of input features 
hidden_size = 4  # Number of nodes in each hidden layer
output_size = 2  # Number of output nodes
deep_feedforward_model = DeepFeedforwardNN(input_size, hidden_size, output_size)

# Define a sample input
sample_input = torch.tensor([[0.5, 0.3, 0.8]])  # Example Data

# Forward pass to get the output
output_deep_feedforward = deep_feedforward_model(sample_input)

# Print the model architecture and output
print(deep_feedforward_model)
print("Output:", output_deep_feedforward)

DeepFeedforwardNN(
  (input_layer): Linear(in_features=3, out_features=4, bias=True)
  (hidden_layers): ModuleList(
    (0-1): 2 x Linear(in_features=4, out_features=4, bias=True)
  )
  (output_layer): Linear(in_features=4, out_features=2, bias=True)
)
Output: tensor([[0.4105, 0.5895]], grad_fn=<SigmoidBackward0>)


# Long Short Term Memory (LSTM)

In [14]:
# LSTM Neural Network
class LSTMNN(BaseNN):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMNN, self).__init__(input_size, hidden_size, output_size)
        self.lstm_layer = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_t, c_t) = self.lstm_layer(x)
        output = torch.sigmoid(self.output_layer(h_t[-1, :, :]))  # Taking the output from the last time step
        return output

In [15]:
# Instantiate the LSTM neural network
input_size = 3  # Number of input features 
hidden_size = 3  # Number of memory cells
output_size = 4  # Number of output nodes
lstm_model = LSTMNN(input_size, hidden_size, output_size)

# Define a sample input
sample_input = torch.rand((1, 4, input_size))  # Example Data

# Forward pass to get the output
output_lstm = lstm_model(sample_input)

# Print the model architecture and output
print(lstm_model)
print("Output:", output_lstm)

LSTMNN(
  (lstm_layer): LSTM(3, 3, batch_first=True)
  (output_layer): Linear(in_features=3, out_features=4, bias=True)
)
Output: tensor([[0.4899, 0.5009, 0.4056, 0.5740]], grad_fn=<SigmoidBackward0>)


# Gated Recurrent Unit (GRU)

In [16]:
class GRUNN(BaseNN):
    def __init__(self, input_size, hidden_size, output_size=1):
        super(GRUNN, self).__init__(input_size, hidden_size, output_size)
        self.gru_layer = nn.GRU(input_size, hidden_size, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, self.output_size)

    def forward(self, x):
        h_t, _ = self.gru_layer(x)
        output = torch.sigmoid(self.output_layer(h_t[:, -1, :]))  # Taking the output from the last time step
        return output

In [17]:
# Instantiate the GRU neural network
input_size = 3  # Number of input features 
hidden_size = 3  # Number of memory cells
output_size = 4  # Number of output nodes
gru_model = GRUNN(input_size, hidden_size, output_size)

# Define a sample input
sample_input = torch.rand((1, 4, input_size))  # Example Data

# Forward pass to get the output
output_gru = gru_model(sample_input)

# Print the model architecture and output
print(gru_model)
print("Output:", output_gru)

GRUNN(
  (gru_layer): GRU(3, 3, batch_first=True)
  (output_layer): Linear(in_features=3, out_features=4, bias=True)
)
Output: tensor([[0.2902, 0.5306, 0.5113, 0.4946]], grad_fn=<SigmoidBackward0>)


# Auto Encoder (AE)

In [18]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Linear(input_size, hidden_size)
        self.decoder = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        encoded = torch.relu(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return decoded

In [19]:
# Instantiate the autoencoder
input_size = 10  # Number of input features
hidden_size = 5  # Number of hidden nodes (compressed representation)
autoencoder = Autoencoder(input_size, hidden_size)

# Define a sample input
sample_input = torch.rand((1, input_size))

# Forward pass to get the reconstructed output
output_autoencoder = autoencoder(sample_input)

# Print the model architecture and output
print(autoencoder)
print("Output:", output_autoencoder)

Autoencoder(
  (encoder): Linear(in_features=10, out_features=5, bias=True)
  (decoder): Linear(in_features=5, out_features=10, bias=True)
)
Output: tensor([[0.4823, 0.4569, 0.4306, 0.4428, 0.5675, 0.3550, 0.4281, 0.5221, 0.4918,
         0.4470]], grad_fn=<SigmoidBackward0>)


# Variational AE (VAE)

In [20]:
class VariationalAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(VariationalAutoencoder, self).__init__()

        # Encoder layers
        self.encoder_fc1 = nn.Linear(input_size, hidden_size)
        self.encoder_fc2_mean = nn.Linear(hidden_size, hidden_size)
        self.encoder_fc2_logvar = nn.Linear(hidden_size, hidden_size)

        # Decoder layers
        self.decoder_fc1 = nn.Linear(hidden_size, input_size)
        self.decoder_fc2 = nn.Linear(input_size, input_size)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        # Encoder
        x = torch.relu(self.encoder_fc1(x))
        mean = self.encoder_fc2_mean(x)
        logvar = self.encoder_fc2_logvar(x)

        # Reparameterization trick
        z = self.reparameterize(mean, logvar)

        # Decoder
        x_hat = torch.relu(self.decoder_fc1(z))
        x_hat = torch.sigmoid(self.decoder_fc2(x_hat))

        return x_hat, mean, logvar

In [21]:
# Instantiate the variational autoencoder
input_size = 4  # Number of input features
hidden_size = 4  # Number of hidden nodes in probabilistic layer
vae = VariationalAutoencoder(input_size, hidden_size)

# Define a sample input
sample_input = torch.rand((1, input_size))

# Forward pass to get the reconstructed output and latent variables
output_vae, mean, logvar = vae(sample_input)

# Print the model architecture and output
print(vae)
print("Output:", output_vae)
print("Mean:", mean)
print("Log Variance:", logvar)

VariationalAutoencoder(
  (encoder_fc1): Linear(in_features=4, out_features=4, bias=True)
  (encoder_fc2_mean): Linear(in_features=4, out_features=4, bias=True)
  (encoder_fc2_logvar): Linear(in_features=4, out_features=4, bias=True)
  (decoder_fc1): Linear(in_features=4, out_features=4, bias=True)
  (decoder_fc2): Linear(in_features=4, out_features=4, bias=True)
)
Output: tensor([[0.4330, 0.5338, 0.3577, 0.3295]], grad_fn=<SigmoidBackward0>)
Mean: tensor([[-0.3101, -0.2446,  0.1739, -0.5934]], grad_fn=<AddmmBackward0>)
Log Variance: tensor([[-0.1155,  0.2399, -0.5895, -0.4484]], grad_fn=<AddmmBackward0>)
